In [4]:
import pandas as pd

import statistics

import random
random.seed(10)


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
from scipy.stats import uniform
import scipy.stats as st
from scipy.stats import pointbiserialr
from scipy.stats import wilcoxon
from scipy.stats import kruskal



import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import GammaRegressor
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor


from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Data Assets


In [20]:
df_balance = pd.read_csv('balancesheet.csv')
df_stock = pd.read_csv('companystockinfo.csv')
df_income = pd.read_csv('incomestatement.csv')
df_fxrate = pd.read_csv('fxrate.csv')
df_forecast = pd.read_csv('forecast.csv')
df_demo = pd.read_csv('sample_output.csv')

# Ouput DataFrame Generation

In [21]:
list_output_columns = df_demo.columns.to_list()

In [22]:
list_output_columns = df_demo.columns.to_list()
dfout = pd.DataFrame(data = None, columns = list_output_columns)
dfout # this is our output dataframe to be filled 

,companyid,latestfinyear,roe fy-4,roe fy-3,roe fy-2,roe fy-1,roe fy,roe fy+1,roe fy+2,roe fy+3,5yr median roe,roa fy-4,roa fy-3,roa fy-2,roa fy-1,roa fy,roa fy+1,roa fy+2,roa fy+3,5yr median roa,netde fy,revenue fy-4,revenue fy-3,revenue fy-2,revenue fy-1,revenue fy,revenue fy+1,revenue fy+2,revenue fy+3,revenue 2yr cagr,revenue 3yr cagr


# Data Generation and Input to dfout

In [23]:
# Inserting company id and latestfinyear

dfout.companyid = df_stock.companyid
dfout.latestfinyear = df_stock.latestfinyear

In [24]:
# Inserting roe fy-4 to fy+3

list_roe = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    list_years = range(year-4, year+4, 1) # generates list of years for which features are required
    
#########################################################################################################
        

# Process for generating roe and roA - 

#Generation of both variables requires profit, equity and revenue data from multiple sheets. 
#When extracting said data for years post the latestfinyear.
#The data needs to be extracted from the forecast dataframe rather than original income or balance sheet dataframe. 


        
    for i in list_years:
        
        search_year = i # year for which values will be lookedup in other dataframes
        
        ############################################################
        ############################################################
        # Extracting variables values for roe 
        ############################################################
        ############################################################
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting net_profit
        
        
        dataframe = df_income
        net_profit = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].netprofit
        
        net_profit_status = net_profit.empty
        
        if net_profit_status == True:
            
            net_profit = 0
            
        net_profit = float(net_profit)
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity
        
        dataframe = df_balance
        total_equity = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totequity
        
        total_equity_status = total_equity.empty
        
        if total_equity_status == True:
            
            total_equity = 0
            
        total_equity = float(total_equity)

        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity_prev_year
        
        dataframe = df_balance
        total_equity_prev_year = dataframe[(dataframe.companyid == companyid) & (dataframe.year == (search_year - 1))].totequity
        
        total_equity_prev_year_status = total_equity_prev_year.empty
        
        if total_equity_prev_year_status == True:
            
            total_equity_prev_year = 0
            
        total_equity_prev_year = float(total_equity_prev_year)
            
        
        ############################################################
        ############################################################
        # Generating the roe
        ############################################################
        ############################################################
        
        # roe = 100 * (netprofit(year) / ((totalequity(year)+totalequity(year-1))/2))
        
        
        if net_profit_status == False:
            
            num = net_profit
            den = (total_equity + total_equity_prev_year) * 0.5
        
            roe = (num/den) * 100
            
        if net_profit_status == True:
            
            dataframe = df_forecast
            roe = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].roe
            
            if roe.empty == True:
                roe = 0
            
            roe = float(roe) # required for all roe directly taken from df_forecasts
        
        ############################################################
        ############################################################
        # Output operations
        ############################################################
        ############################################################
        
        
        '''
        print('search_year', search_year)
        print('companyid',companyid)
        print(list_years)
        print(search_year)
        print(roe)
        print('Roe generated for year', search_year)
        print('companyid', companyid)
        print('Roe generated for year', search_year)     
        print('toteqprev', total_equity_prev_year)
        print(search_year)
        print('companyid', companyid)
        print(search_year)
        print(roe)
        '''
        
        # Inserting roe into dfout
        
        list_roe.append(roe)
        

########## Dividing roe list into chunks of 8 

# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
# How many elements each
# list should have
n = 8
 
list_roe_chunks = list(divide_chunks(list_roe, n))  


########## Inserting roe values in dfout

for i in dfout.index:
    
    chunk = list_roe_chunks[i]
    
    dfout['roe fy-4'][i] = chunk[0]
    dfout['roe fy-3'][i] = chunk[1]
    dfout['roe fy-2'][i] = chunk[2]
    dfout['roe fy-1'][i] = chunk[3]
    dfout['roe fy'][i]   = chunk[4]
    dfout['roe fy+1'][i] = chunk[5]
    dfout['roe fy+2'][i] = chunk[6]
    dfout['roe fy+3'][i] = chunk[7]
    
        
            
        

In [33]:
############ Inserting roa fy-4 to fy+3

list_roa = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    list_years = range(year-4, year+4, 1) # generates list of years for which features are required
    
#########################################################################################################
        

# Process for generating roe and roA - 

#Generation of both variables requires profit, equity and revenue data from multiple sheets. 
#When extracting said data for years post the latestfinyear.
#The data needs to be extracted from the forecast dataframe rather than original income or balance sheet dataframe. 


        
    for i in list_years:
        
        search_year = i # year for which values will be lookedup in other dataframes
        
        ############################################################
        ############################################################
        # Extracting variables values for roe 
        ############################################################
        ############################################################
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting net_profit
        
        
        dataframe = df_income
        net_profit = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].netprofit
        
        net_profit_status = net_profit.empty
        
        if net_profit_status == True:
            
            net_profit = 0
            
        net_profit = float(net_profit)
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity
        
        dataframe = df_balance
        total_assets = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totassets
        
        total_assets_status = total_assets.empty
        
        if total_assets_status == True:
            
            total_assets = 0
            
        total_assets = float(total_assets)

        #___________________________________________________________
        #___________________________________________________________
        # Extracting total_equity_prev_year
        
        dataframe = df_balance
        total_assets_prev_year = dataframe[(dataframe.companyid == companyid) & (dataframe.year == (search_year - 1))].totassets
        
        total_assets_prev_year_status = total_assets_prev_year.empty
        
        if total_assets_prev_year_status == True:
            
            total_assets_prev_year = 0
            
        total_assets_prev_year = float(total_assets_prev_year)
            
        
        ############################################################
        ############################################################
        # Generating the roe
        ############################################################
        ############################################################
        
        # roe = 100 * (netprofit(year) / ((totalassets(year)+totalassets(year-1))/2))
        
        
        if net_profit_status == False:
            
            num = net_profit
            den = (total_assets + total_assets_prev_year) * 0.5
        
            roa = (num/den) * 100
            
        if net_profit_status == True:
            
            dataframe = df_forecast
            roa = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].roa
            
            if roa.empty == True:
                roa = 0
            
            roa = float(roa) # required for all roe directly taken from df_forecasts
        
        ############################################################
        ############################################################
        # Output operations
        ############################################################
        ############################################################
        
        
        '''
        
        print('companyid',companyid)
        print(list_years)
        print('Roe generated for year', search_year)
        print('companyid', companyid)
        print('Roe generated for year', search_year)     
        print('toteqprev', total_equity_prev_year)
        
        print('companyid', companyid)
        print(search_year)
        
        print('companyid:', companyid)
        print('latestinfyear:', year)
        print('search_year:', search_year)
        print('roa:',roa)
        print('\n')
        
        
        '''
        
        # Inserting roe into dfout
       
        list_roa.append(roa)
        
     

######### Dividing roa list into chunks of 8 

# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
# How many elements each
# list should have
n = 8
 
list_roa_chunks = list(divide_chunks(list_roa, n))  


########## Inserting roe values in dfout

for i in dfout.index:
    
    chunk = list_roa_chunks[i]
    
    dfout['roa fy-4'][i] = chunk[0]
    dfout['roa fy-3'][i] = chunk[1]
    dfout['roa fy-2'][i] = chunk[2]
    dfout['roa fy-1'][i] = chunk[3]
    dfout['roa fy'][i]   = chunk[4]
    dfout['roa fy+1'][i] = chunk[5]
    dfout['roa fy+2'][i] = chunk[6]
    dfout['roa fy+3'][i] = chunk[7]
   

In [37]:
### Generating 5ry median roe and roa

# missing value imputation is required for roe and roa columns
# then columns need to be typecasted
# only then numpy computations will be enabled

### imputation
roe_columns = ['roe fy-4', 'roe fy-3', 'roe fy-2', 'roe fy-1', 'roe fy', 'roe fy+1', 'roe fy+2', 'roe fy+3']
roa_columns = ['roa fy-4', 'roa fy-3', 'roa fy-2', 'roa fy-1', 'roa fy', 'roa fy+1', 'roa fy+2', 'roa fy+3']

# imputation value is set to 0.01 to recognize the observations and not affect statistical computation
#dfout[roe_columns] = dfout[roe_columns].fillna(value = 0.01)
#dfout[roa_columns] = dfout[roa_columns].fillna(value = 0.01)

### type casting
# successful imputation will automatically typecast the columns
# if not, then manual typcasting will be required

for i in dfout.index:
    
    # inserting median for roe
    list_values = dfout[['roe fy-4', 'roe fy-3', 'roe fy-2', 'roe fy-1', 'roe fy']].iloc[i].to_list()
    median = statistics.median(list_values)
    dfout['5yr median roe'][i] = median
    
    # inserting median for roa
    list_values = dfout[['roa fy-4', 'roa fy-3', 'roa fy-2', 'roa fy-1', 'roa fy']].iloc[i].to_list()
    median = statistics.median(list_values)
    dfout['5yr median roa'][i] = median

In [26]:
### generating revenue columns
# similar procedure to roe and roa


list_revenue = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    list_years = range(year-4, year+4, 1) # generates list of years for which features are required
    
#########################################################################################################
        

# Process for generating roe and roA - 

#Generation of both variables requires profit, equity and revenue data from multiple sheets. 
#When extracting said data for years post the latestfinyear.
#The data needs to be extracted from the forecast dataframe rather than original income or balance sheet dataframe. 


        
    for i in list_years:
        
        search_year = i # year for which values will be lookedup in other dataframes
        
        ############################################################
        ############################################################
        # Extracting variables values for roe 
        ############################################################
        ############################################################
        
        #___________________________________________________________
        #___________________________________________________________
        # Extracting net_profit
        
        
        dataframe = df_income
        revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue
        
        revenue_status = revenue.empty
        
        if revenue_status == True: # to check if revenue was missing in df_income
            
            # if revenue missing in df_income; collect revenue from forecast
            
            dataframe = df_forecast
            revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue

            if (revenue.empty == True):
                
                revenue = 0.01 # if revenue is not found in forecast, value/observation is an indentifier
            
        revenue = float(revenue)
        
        ####### An additional operation on revenue is required based on currency
        
        dataframe = df_stock
        currency_reporting = dataframe[dataframe.companyid == companyid].reportingcurr
        
        #if currency_reporting.empty == True : 
            #currency_reporting = 0.0
            
        x = currency_reporting.to_string()
        x = x.split(' ')
        x = x[(len(x)-1)]
        currency_reporting = x
        
        #__________
        
        dataframe = df_stock
        currency_estimated = dataframe[dataframe.companyid == companyid].estcurr
        
        if currency_estimated.empty == True : 
            currency_estimated = 'Missing'
            
        x = currency_estimated.to_string()
        x = x.split(' ')
        x = x[(len(x)-1)]
        currency_estimated = x        
        

        #__________
        
        # for rate extraction; 
        # equivalency from fx to stock datasheets
        # base currency is considered = reporting currency
        # quot currency is considered = est currency
        
        # Logic
        
        # 1. 
        
        dataframe = df_fxrate
        rate = dataframe[(dataframe.basecur == currency_reporting) & (dataframe.quotcur == currency_estimated)].rate
        
        if rate.empty == True :
            
            rate = 0.01 # place holder if no rate is found from the fx dataframe
            
        rate = float(rate)
        
        # If reporting currency is different than estimated currency - df_stock
        
        if currency_reporting != currency_estimated:
            
            revenue = revenue * rate
            
        else:
            
            revenue = revenue
        
        
        ############################################################
        ############################################################
        # Output operations
        ############################################################
        ############################################################        
        
                       
        '''
        
        print('companyid',companyid)
        print(list_years)
        print('Roe generated for year', search_year)
        print('companyid', companyid)
        print('Roe generated for year', search_year)     
        print('toteqprev', total_equity_prev_year)
        print('companyid', companyid)
        print(search_year)
        print('companyid:', companyid)
        print('latestinfyear:', year)
        print('search_year:', search_year)
        print('revenue:',revenue)
        print('\n')
        '''
        
        list_revenue.append(revenue)
        
        
       
########## Dividing revenue list into chunks of 8 

# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
     
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
 
# How many elements each
# list should have
n = 8
 
list_revenue_chunks = list(divide_chunks(list_revenue, n))  


########## Inserting roe values in dfout

for i in dfout.index:
    
    chunk = list_revenue_chunks[i]
    
    dfout['revenue fy-4'][i] = chunk[0]
    dfout['revenue fy-3'][i] = chunk[1]
    dfout['revenue fy-2'][i] = chunk[2]
    dfout['revenue fy-1'][i] = chunk[3]
    dfout['revenue fy'][i]   = chunk[4]
    dfout['revenue fy+1'][i] = chunk[5]
    dfout['revenue fy+2'][i] = chunk[6]
    dfout['revenue fy+3'][i] = chunk[7]
        


In [27]:
dataframe = df_income
revenue = dataframe[(dataframe.companyid == 169) & (dataframe.year == 2016)].revenue
revenue
        

1085    7075.0
Name: revenue, dtype: float64

In [28]:
# Genering netde for latestfinyear

list_de = []

list_revenue = [] # Global roe list for all 8 years, stored sequentially

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    year = dfout.latestfinyear[i] # collectes the year for current row
    
    search_year = year
    
    de = 0 #prior to search, overwrite when de is calculated
    
    dataframe = df_balance
    total_debt = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totaldebt
    
    dataframe = df_balance
    cash_and_cash_equivalent = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].cashncashequiv
    
    dataframe = df_balance
    total_equity = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].totequity
    
    
    # Check if any of the numerica values come out empty or missing
    
    if total_debt.empty == True:
        
        de = 0.01
        
    if cash_and_cash_equivalent.empty == True:
        
        de = 0.01
        
    if total_equity.empty == True:
        
        de = 0.01
        
    
    total_debt = float(total_debt)
    cash_and_cash_equivalent = float(cash_and_cash_equivalent)
    total_equity = float(total_equity)
    
    # Formulae
    # netde[2020] = (totaldebt[2020] – cashncashequiv[2020])/ totequity[2020]
    
    de = (total_debt - cash_and_cash_equivalent) / total_equity
    
    list_de.append(de)


####################
# Add de values to df out

dfout['netde fy'] = list_de

In [29]:
# Generating revenue 2 yr cagr

################## generating revenue columns
# similar procedure to roe and roa


list_cagr_2year = [] 


for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    search_year = dfout.latestfinyear[i] # collectes the year for current row
    
    search_year_2 = search_year + 2
    
    
    cagr_2year = 0

    
    # Extracting values for cagr computation
    
    # Extracting revenue for latest year + 2 from forecast
    
    dataframe = df_forecast
    revenue_2 = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year_2)].revenue
    
  
    # Extracting revenue for latest year from balance sheet dataframe
    
    dataframe = df_income
    revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue
    
    #print('2yrev: ', revenue_2)
    #print('rev: ', revenue)
    #print('\n')
    #___________________________________
    
    if (revenue_2.empty == True) | (revenue.empty == True):
        
        cagr_2year = 0.01
        
    else:
        
        revenue_2 = float(revenue_2)
        revenue = float(revenue)
        
        cagr_2year = 100 * ((revenue_2 / revenue) ** (0.5) - 1)
    
    list_cagr_2year.append(cagr_2year)
    
    
dfout['revenue 2yr cagr'] = list_cagr_2year # inserting list into the column

    
#########################################################################################################

# Generating revenue 2 yr cagr

#revenue 3yr cagr = 100*[(revenue fy+3 / revenue fy)^(0.33) – 1]





In [30]:
# Generating revenue 3 yr cagr

################## generating revenue columns
# similar procedure to roe and roa



list_cagr_3year = []

for i in dfout.index:
    
    companyid = dfout.companyid[i]
    
    search_year = dfout.latestfinyear[i] # collectes the year for current row
    
    search_year_3 = search_year + 3
    
    cagr_3year = 0
    
    # Extracting values for cagr computation
   
    # Extracting revenue for latest year + 3 from forecast
    
    dataframe = df_forecast
    revenue_3 = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year_3)].revenue
    
    # Extracting revenue for latest year from balance sheet dataframe
    
    dataframe = df_income
    revenue = dataframe[(dataframe.companyid == companyid) & (dataframe.year == search_year)].revenue
    
    #print('2yrev: ', revenue_2)
    #print('rev: ', revenue)
    #print('\n')
    
    #___________________________
    
    if (revenue_3.empty == True) | (revenue.empty == True):
        
        cagr_3year = 0.01
        
    else:
        
        revenue_3 = float(revenue_3)
        revenue = float(revenue)
        
        cagr_3year = 100 * ((revenue_3 / revenue) ** (0.33) - 1)
    
    list_cagr_3year.append(cagr_3year)
    
    

dfout['revenue 3yr cagr'] = list_cagr_3year # inserting list into the column
    
#########################################################################################################

# Generating revenue 2 yr cagr

#revenue 3yr cagr = 100*[(revenue fy+3 / revenue fy)^(0.33) – 1]





# dfout display

In [38]:
dfout

,companyid,latestfinyear,roe fy-4,roe fy-3,roe fy-2,roe fy-1,roe fy,roe fy+1,roe fy+2,roe fy+3,5yr median roe,roa fy-4,roa fy-3,roa fy-2,roa fy-1,roa fy,roa fy+1,roa fy+2,roa fy+3,5yr median roa,netde fy,revenue fy-4,revenue fy-3,revenue fy-2,revenue fy-1,revenue fy,revenue fy+1,revenue fy+2,revenue fy+3,revenue 2yr cagr,revenue 3yr cagr
0,30,2020,22.3,18.4,15.6,19.8,21.1,19.3,18.5,16.4,19.8,18.1,15,13,17,14.3,NaN,NaN,NaN,15,2.908e-01,4.69e+06,4.88e+06,5.47e+06,6.22e+06,5.97e+06,6.17e+06,6.67e+06,7.18e+06,5.747,6.310
1,156,2020,8.45,8.98,8.87,7.91,-2.87,0,0,0,8.45,7.27,7.6,7.2,6.22,-2.15,0,0,0,7.2,2.870e-01,1.8e+03,1.92e+03,2.07e+03,2.41e+03,923,0.01,0.01,0.01,0.010,0.010
2,20,2020,11.3,8.86,8.97,5.89,-3.03,-1.14,1.91,3.55,8.86,7.02,6.15,6.79,4.38,-2.2,-0.24,1.84,3.12,6.15,8.751e-02,4.8e+06,4.2e+06,4.22e+06,4.05e+06,2.05e+06,2.1e+06,2.63e+06,2.95e+06,13.380,12.848
3,169,2020,22.5,15,24.3,23.2,16.3,12.9,13.8,14,22.5,14.4,9.93,15.9,12.3,8.8,7.64,8.33,8.73,12.3,1.909e-01,7.08e+03,7.77e+03,8.42e+03,8.78e+03,8.26e+03,1.02e+04,1.14e+04,1.25e+04,17.355,14.725
4,37,2021,96.2,15.6,44,27.6,15.9,15.2,14.9,15.6,27.6,74.1,10.7,22.4,13.9,10.1,13.9,13.9,16.6,13.9,-3.707e-01,550,542,681,664,626,679,728,823,7.851,9.460
5,180,2020,36.1,36.6,31.9,31.2,30.2,29.3,30.7,31.1,31.9,10.7,11,9.82,10,9.6,10.3,11.4,12.1,10,2.506e-01,1.72e+05,1.86e+05,1.92e+05,2.1e+05,2.18e+05,2.27e+05,2.46e+05,2.65e+05,6.280,6.623
6,22,2020,6.08,5.23,3.18,3.66,1.16,0.3,1.6,3.8,3.66,2.7,2.39,1.49,1.73,0.504,0.1,0.7,1.6,1.73,6.032e-01,2.28e+06,2.2e+06,2.55e+06,2.74e+06,2.89e+06,2.52e+06,2.59e+06,2.78e+06,-5.355,-1.265
7,124,2021,18.6,11,6.72,2.14,7.93,14.6,17.4,20.8,7.93,7.89,4.57,2.8,0.93,3.7,NaN,NaN,NaN,3.7,4.064e-01,1.73e+04,1.71e+04,1.72e+04,1.61e+04,1.65e+04,1.95e+04,2.28e+04,2.64e+04,17.409,16.782
8,171,2021,3,0.309,5.54,-5.6,13.5,13.4,13.5,13.6,3,1.49,0.147,2.5,-1.88,3.94,5,5.04,5.28,1.49,1.655e+00,605,640,676,633,717,808,880,949,10.750,9.671
9,17,2021,11.3,12.5,12.5,9.44,5.42,9.95,11.6,0,11.3,5.89,6.6,6.66,5.15,3.1,NaN,NaN,0,5.89,4.024e-01,2.76e+04,3.1e+04,3.57e+04,3.58e+04,3.01e+04,3.63e+04,4.09e+04,0.01,16.562,0.010


# dfoutexport



In [39]:
dfout.to_csv('dfout.csv')

# Testing

In [16]:
dataframe = df_income
dataframe[(dataframe.companyid == companyid) & (dataframe.year == year)]


AttributeError: 'DataFrame' object has no attribute 'net'

In [ ]:
len(dfout)

# Reference DFs

In [ ]:
df_balance[(df_balance.companyid == 172) & (df_balance.year == 2019)]

In [ ]:
df_income[df_income.companyid == 163]

In [ ]:
df_income.info()

In [ ]:
df_demo

In [ ]:
df_forecast[df_forecast.companyid == 30]

In [ ]:
dataframe = df_forecast
revenue_from_forecast = dataframe[(dataframe.companyid == 30) & (dataframe.year == 2021)].revenue
revenue_from_forecast

In [ ]:
df_fxrate[df_fxrate.basecur == 'CNY']

In [ ]:
df_stock

In [ ]:
df_stock.isna().sum()

In [ ]:
df_fxrate.isna().sum()